In [17]:
import face_recognition
import cv2
import numpy as np
import skimage 
import glob

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Load a sample picture and learn how to recognize it.
aidar_image = face_recognition.load_image_file("aidar.jpg")
aidar_face_encoding = face_recognition.face_encodings(aidar_image, num_jitters=4)[0]

# Load a second sample picture and learn how to recognize it.
serik_image = face_recognition.load_image_file("serik.jpg")
serik_face_encoding = face_recognition.face_encodings(serik_image, num_jitters=4)[0]

# Load a third sample picture and learn how to recognize it.
askhat_image = face_recognition.load_image_file("askhat.jpg")
askhat_face_encoding = face_recognition.face_encodings(askhat_image, num_jitters=4)[0]


# Create arrays of known face encodings and their names
known_face_encodings = [
    aidar_face_encoding,
    serik_face_encoding,
    askhat_face_encoding
]
known_face_names = [
    "Aidar",
    "Serik",
    "Askhat"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
test_images = []
for filename in glob.glob('test/*.jpg'): #assuming gif
    test_images.append(filename)

print(test_images)


['test/aibek.jpg', 'test/aibek2.jpg', 'test/aidar.jpg', 'test/aidar1.jpg', 'test/banu.jpg', 'test/serik1.jpg', 'test/serik2.jpg']


In [18]:
for file_path in test_images:
    # Grab a single frame of video
    frame = cv2.imread(file_path)

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)
            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame
    print(str(face_names) + " " + str(file_path))


['Unknown'] test/aibek.jpg
['Unknown'] test/aibek2.jpg
['Aidar'] test/aidar.jpg
['Aidar'] test/aidar1.jpg
['Unknown'] test/banu.jpg
['Unknown'] test/serik1.jpg
['Serik'] test/serik2.jpg
